In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")

state = "n : ℕ\n⊢ gcd n n = n"
tokenized_state = tokenizer(state, return_tensors="pt")

# Generate a single tactic.
tactic_ids = model.generate(tokenized_state.input_ids, max_length=1024)
tactic = tokenizer.decode(tactic_ids[0], skip_special_tokens=True)
print(tactic, end="\n\n")

# Generate multiple tactics via beam search.
tactic_candidates_ids = model.generate(
    tokenized_state.input_ids,
    max_length=1024,
    num_beams=4,
    length_penalty=0.0,
    do_sample=False,
    num_return_sequences=4,
    early_stopping=False,
)
tactic_candidates = tokenizer.batch_decode(
    tactic_candidates_ids, skip_special_tokens=True
)
for tac in tactic_candidates:
    print(tac)

tokenizer_config.json:   0%|          | 0.00/25.6k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

rw [<a>Nat.gcd_comm</a>]

rw [<a>Nat.gcd_comm</a>]
rw [<a>Nat.gcd_comm</a>, <a>Nat.gcd_1</a>]
rw [<a>Nat.gcd_comm</a>, <a>Nat.gcd_rec</a>]
rw [<a>Nat.gcd_comm</a>, <a>Nat.gcd_gcd_self_left_right</a>]


In [2]:
!ls

/nethome/hsun409/anaconda3/envs/dojo/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


/bin/bash: /nethome/hsun409/anaconda3/envs/dojo/lib/libtinfo.so.6: no version information available (required by /bin/bash)
benchmark.ipynb  generator  LICENSE   prover	 retrieval
common.py	 images     mypy.ini  README.md  scripts


In [3]:
import torch

In [8]:
ckpt_path = 'leandojo-pl-ckpts/generator_random.ckpt/checkpoint/mp_rank_00_model_states.pt'
ckpt = torch.load(ckpt_path, map_location=torch.device('cpu'))
ckpt.keys()


dict_keys(['module', 'buffer_names', 'optimizer', 'param_shapes', 'frozen_param_shapes', 'shared_params', 'frozen_param_fragments', 'lr_scheduler', 'data_sampler', 'random_ltd', 'sparse_tensor_module_names', 'skipped_steps', 'global_steps', 'global_samples', 'dp_world_size', 'mp_world_size', 'ds_config', 'ds_version', 'epoch', 'global_step', 'pytorch-lightning_version', 'loops', 'callbacks', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])

In [22]:
import os
import tempfile
from pytorch_lightning.utilities.deepspeed import (
    convert_zero_checkpoint_to_fp32_state_dict,
)

In [19]:
# Instead of pointing directly to the model states file, point to the parent directory.
ckpt_path = 'leandojo-pl-ckpts/generator_random.ckpt'


def _is_deepspeed_checkpoint(path: str):
    if not os.path.exists(path):
        raise FileExistsError(f"Checkpoint {path} does not exist.")
    return os.path.isdir(path) and os.path.exists(os.path.join(path, "zero_to_fp32.py"))

In [20]:
_is_deepspeed_checkpoint(ckpt_path)

True

In [23]:
with tempfile.TemporaryDirectory() as dirname:
    path = os.path.join(dirname, "lightning.cpkt")
    convert_zero_checkpoint_to_fp32_state_dict(ckpt_path, path)

[2024-05-01 20:26:13,920] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/nethome/hsun409/anaconda3/envs/dojo/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.1
 [WARNING]  using untested triton version (2.1.0), only 1.0.0 is known to be compatible


FileNotFoundError: Directory 'leandojo-pl-ckpts/generator_random.ckpt/version https://git-lfs.github.com/spec/v1
oid sha256:47320987f9a49d5b00119b960f247a956773f57543982b8bfcb6da5bb3afd9ef
size 10' doesn't exist

In [24]:
import os
import tempfile
from pytorch_lightning.utilities.deepspeed import convert_zero_checkpoint_to_fp32_state_dict

# Instead of pointing directly to the model states file, point to the parent directory.
ckpt_path = 'leandojo-pl-ckpts/generator_random.ckpt/checkpoint'

# Verify that the checkpoint directory is correctly structured and exists
def _is_deepspeed_checkpoint(path: str):
    required_files = ["zero_to_fp32.py", "latest"]
    exists = all(os.path.exists(os.path.join(path, f)) for f in required_files)
    if not exists:
        raise FileNotFoundError(f"Required files are missing in the directory {path}")
    return exists

try:
    if _is_deepspeed_checkpoint(ckpt_path):
        with tempfile.TemporaryDirectory() as dirname:
            path = os.path.join(dirname, "lightning.ckpt")
            # Convert the Zero checkpoint to a full precision state dict for loading
            convert_zero_checkpoint_to_fp32_state_dict(ckpt_path, path)
            print(f"Checkpoint converted and saved to {path}")
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: Required files are missing in the directory leandojo-pl-ckpts/generator_random.ckpt/checkpoint


In [25]:
ckpt_path = 'leandojo-pl-ckpts/generator_random.ckpt/checkpoint'

def _is_deepspeed_checkpoint(path: str):
    if not os.path.exists(path):
        raise FileExistsError(f"Checkpoint {path} does not exist.")
    return os.path.isdir(path) and os.path.exists(os.path.join(path, "zero_to_fp32.py"))

with tempfile.TemporaryDirectory() as dirname:
    path = os.path.join(dirname, "lightning.cpkt")
    convert_zero_checkpoint_to_fp32_state_dict(ckpt_path, path)

ValueError: Unable to find 'latest' file at leandojo-pl-ckpts/generator_random.ckpt/checkpoint/latest

In [26]:
ckpt_path = 'leandojo-pl-ckpts/generator_random.ckpt'

def _is_deepspeed_checkpoint(path: str):
    if not os.path.exists(path):
        raise FileExistsError(f"Checkpoint {path} does not exist.")
    return os.path.isdir(path) and os.path.exists(os.path.join(path, "zero_to_fp32.py"))

with tempfile.TemporaryDirectory() as dirname:
    path = os.path.join(dirname, "lightning.cpkt")
    convert_zero_checkpoint_to_fp32_state_dict(ckpt_path, path)

FileNotFoundError: Directory 'leandojo-pl-ckpts/generator_random.ckpt/version https://git-lfs.github.com/spec/v1
oid sha256:47320987f9a49d5b00119b960f247a956773f57543982b8bfcb6da5bb3afd9ef
size 10' doesn't exist

In [27]:
ckpt_path = 'leandojo-pl-ckpts/generator_random.ckpt'
tag = 'checkpoint'

def _is_deepspeed_checkpoint(path: str):
    if not os.path.exists(path):
        raise FileExistsError(f"Checkpoint {path} does not exist.")
    return os.path.isdir(path) and os.path.exists(os.path.join(path, "zero_to_fp32.py"))

with tempfile.TemporaryDirectory() as dirname:
    path = os.path.join(dirname, "lightning.cpkt")
    convert_zero_checkpoint_to_fp32_state_dict(ckpt_path, path, tag=tag)

Processing zero checkpoint 'leandojo-pl-ckpts/generator_random.ckpt/checkpoint'
Detected checkpoint of type zero stage 2, world_size: 1
Parsing checkpoint created by deepspeed==0.14.0
Reconstructed fp32 state dict with 170 params 299637760 elements


FileNotFoundError: Directory '<function ds_checkpoint_dir at 0x7fe6191a3640>' doesn't exist

In [28]:
ckpt_path = 'leandojo-pl-ckpts/generator_random.ckpt/checkpoint'

def _is_deepspeed_checkpoint(path: str):
    if not os.path.exists(path):
        raise FileExistsError(f"Checkpoint {path} does not exist.")
    return os.path.isdir(path) and os.path.exists(os.path.join(path, "zero_to_fp32.py"))

with tempfile.TemporaryDirectory() as dirname:
    path = os.path.join(dirname, "lightning.cpkt")
    convert_zero_checkpoint_to_fp32_state_dict(ckpt_path, path)

ValueError: Unable to find 'latest' file at leandojo-pl-ckpts/generator_random.ckpt/checkpoint/latest

In [29]:
ckpt_path = 'leandojo-pl-ckpts/generator_random.ckpt/checkpoint'

def _is_deepspeed_checkpoint(path: str):
    if not os.path.exists(path):
        raise FileExistsError(f"Checkpoint {path} does not exist.")
    return os.path.isdir(path) and os.path.exists(os.path.join(path, "zero_to_fp32.py"))

with tempfile.TemporaryDirectory() as dirname:
    path = os.path.join(dirname, "lightning.cpkt")
    convert_zero_checkpoint_to_fp32_state_dict(ckpt_path, path, tag='')

Processing zero checkpoint 'leandojo-pl-ckpts/generator_random.ckpt/checkpoint/'
Detected checkpoint of type zero stage 2, world_size: 1
Parsing checkpoint created by deepspeed==0.14.0
Reconstructed fp32 state dict with 170 params 299637760 elements


ValueError: Unable to find 'latest' file at leandojo-pl-ckpts/generator_random.ckpt/checkpoint/latest

In [30]:
ckpt_path = 'leandojo-pl-ckpts/generator_random.ckpt'

def _is_deepspeed_checkpoint(path: str):
    if not os.path.exists(path):
        raise FileExistsError(f"Checkpoint {path} does not exist.")
    return os.path.isdir(path) and os.path.exists(os.path.join(path, "zero_to_fp32.py"))

with tempfile.TemporaryDirectory() as dirname:
    path = os.path.join(dirname, "lightning.cpkt")
    convert_zero_checkpoint_to_fp32_state_dict(ckpt_path, path)

Processing zero checkpoint 'leandojo-pl-ckpts/generator_random.ckpt/checkpoint'
Detected checkpoint of type zero stage 2, world_size: 1
Parsing checkpoint created by deepspeed==0.14.0
Reconstructed fp32 state dict with 170 params 299637760 elements
Saving fp32 state dict to /tmp/tmpy2ayeo8r/lightning.cpkt
